In [10]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from helper_functions_CART import *

### 1. Load data and prepare data

In [4]:
admissions_df = pd.read_csv('../datasets/data/bar_pass_prediction.csv', index_col=False)
print(admissions_df.shape)
print(admissions_df.columns)
print(admissions_df.isnull().sum())
display(admissions_df.head())

(22407, 39)
Index(['decile1b', 'decile3', 'ID', 'decile1', 'sex', 'race', 'cluster',
       'lsat', 'ugpa', 'zfygpa', 'DOB_yr', 'grad', 'zgpa', 'bar1', 'bar1_yr',
       'bar2', 'bar2_yr', 'fulltime', 'fam_inc', 'age', 'gender', 'parttime',
       'male', 'race1', 'race2', 'Dropout', 'other', 'asian', 'black', 'hisp',
       'pass_bar', 'bar', 'bar_passed', 'tier', 'index6040', 'indxgrp',
       'indxgrp2', 'dnn_bar_pass_prediction', 'gpa'],
      dtype='object')
decile1b                   1604
decile3                    1604
ID                            0
decile1                    1092
sex                           5
race                         16
cluster                      96
lsat                          0
ugpa                          0
zfygpa                      984
DOB_yr                       50
grad                          3
zgpa                       1289
bar1                          0
bar1_yr                      39
bar2                          0
bar2_yr             

,decile1b,decile3,ID,decile1,sex,race,cluster,lsat,ugpa,zfygpa,...,hisp,pass_bar,bar,bar_passed,tier,index6040,indxgrp,indxgrp2,dnn_bar_pass_prediction,gpa
0,10.0,10.0,2,10.0,1.0,7.0,1.0,44.0,3.5,1.33,...,0,1,a Passed 1st time,True,4.0,886.842082,g 700+,i 820+,0.979804,3.5
1,5.0,4.0,3,5.0,1.0,7.0,2.0,29.0,3.5,-0.11,...,0,1,a Passed 1st time,True,2.0,649.999987,f 640-700,f 640-700,0.979804,3.5
2,3.0,2.0,36,3.0,2.0,7.0,3.0,36.0,3.5,-0.64,...,0,1,a Passed 1st time,True,3.0,760.526298,g 700+,h 760-820,0.979804,3.5
3,7.0,4.0,52,7.0,2.0,7.0,3.0,39.0,3.5,0.34,...,0,1,a Passed 1st time,True,3.0,807.894717,g 700+,h 760-820,0.979804,3.5
4,9.0,8.0,55,9.0,2.0,7.0,4.0,48.0,3.5,1.02,...,0,1,a Passed 1st time,True,5.0,949.999974,g 700+,i 820+,0.979804,3.5


In [5]:
# Remove all rows with missing values
admissions_df = admissions_df.dropna()
print(admissions_df.shape)
print(admissions_df.isnull().sum())

(20427, 39)
decile1b                   0
decile3                    0
ID                         0
decile1                    0
sex                        0
race                       0
cluster                    0
lsat                       0
ugpa                       0
zfygpa                     0
DOB_yr                     0
grad                       0
zgpa                       0
bar1                       0
bar1_yr                    0
bar2                       0
bar2_yr                    0
fulltime                   0
fam_inc                    0
age                        0
gender                     0
parttime                   0
male                       0
race1                      0
race2                      0
Dropout                    0
other                      0
asian                      0
black                      0
hisp                       0
pass_bar                   0
bar                        0
bar_passed                 0
tier                       0
in

**UI text**

A subset of the [Law School Admission Bar*](https://www.kaggle.com/datasets/danofer/law-school-admissions-bar-passage) dataset is used as a demo. Synthetic data will be generated for the following columns: 

- sex: student gender, i.e. 1 (male), 2 (female)
- race1: race, i.e. asian, black, hispanic, white, other
- ugpa: The student's undergraduate GPA, continous variable;
- bar: Ground truth label indicating whether or not the student passed the bar, i.e. passed 1st time, passed 2nd time, failed, non-graduated

The CART method will be used  evaluate the distribution and correlation differences between the real and synthetic data.

*The original paper can be found [here](https://files.eric.ed.gov/fulltext/ED469370.pdf).

In [ ]:
# demo dataset
real_data = admissions_df[['sex', 'race1', 'ugpa', 'bar']]
real_data.head()

,sex,race1,ugpa,bar
0,1.0,white,3.5,a Passed 1st time
1,1.0,white,3.5,a Passed 1st time
2,2.0,white,3.5,a Passed 1st time
3,2.0,white,3.5,a Passed 1st time
4,2.0,white,3.5,a Passed 1st time


In [7]:
# automatically detect the data type of each column
dtypes_dict = real_data.dtypes.to_dict()
dtypes_dict

{'sex': dtype('float64'),
 'race1': dtype('O'),
 'ugpa': dtype('float64'),
 'bar': dtype('O')}

In [8]:
# replace 'float64' with 'float' and 'O' with 'object'
dtypes_dict = {k: 'float' if v == 'float64' else 'category' if v == 'O' else v for k, v in dtypes_dict.items()}
dtypes_dict

{'sex': 'float', 'race1': 'category', 'ugpa': 'float', 'bar': 'category'}

In [11]:
# Encode string columns to numeric values
label_encoders = {}
for column in real_data.select_dtypes(include=['object']).columns:
    label_encoders[column] = LabelEncoder()
    real_data[column] = label_encoders[column].fit_transform(real_data[column])

real_data.head()

/var/folders/v8/64zc34sn3b95klfm660085h80000gn/T/ipykernel_73471/158703987.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  real_data[column] = label_encoders[column].fit_transform(real_data[column])
/var/folders/v8/64zc34sn3b95klfm660085h80000gn/T/ipykernel_73471/158703987.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  real_data[column] = label_encoders[column].fit_transform(real_data[column])


,sex,race1,ugpa,bar
0,1.0,4,3.5,0
1,1.0,4,3.5,0
2,2.0,4,3.5,0
3,2.0,4,3.5,0
4,2.0,4,3.5,0


### 2. Gaussian Copula

**UI text**

The Gaussian copula (GC) method generates synthetic data by modeling dependencies between variables while preserving their individual distributions. It works in two main steps:
1. The real data is transformed into a uniform distribution. Correlations between variables are modeled using a multivariate normal distribution (referred to as a *Gaussian copula*);
2. Synthetic data is created by sampling from this Gaussian copula and transforming the samples back to the original data distributions. 

The GC approach is especially useful for capturing complex relationships and dependencies while maintaining the characteristics of each variable.

In [12]:
# Initialize synthesizer and fit it to the data
synthesizer = GaussianCopulaSynthesizer()
synthesizer.fit(real_data)

In [13]:
# Generate synthetic data
synthetic_data = synthesizer.sample(1000)

# Output some results
print("Original Data (first 5 rows):\n", real_data.head())
print("\nSynthetic Data (first 5 rows):\n", synthetic_data.head())

Original Data (first 5 rows):
    sex  race1  ugpa  bar
0  1.0      4   3.5    0
1  1.0      4   3.5    0
2  2.0      4   3.5    0
3  2.0      4   3.5    0
4  2.0      4   3.5    0

Synthetic Data (first 5 rows):
    sex  race1  ugpa  bar
0  2.0    4.0   2.8  0.0
1  2.0    4.0   3.5  0.0
2  2.0    4.0   3.1  0.0
3  1.0    1.0   3.5  0.0
4  2.0    2.0   3.4  0.0


### 3. Evaluating produced synthetic data 

In [14]:
run_diagnostic(real_data, synthetic_data, target_column='gpa')

{'distribution_results': {'sex': {'ks_stat': 0.007970676065991111,
   'p_value': 0.9999999678785447},
  'race1': {'ks_stat': 0.07565433005336075, 'p_value': 3.438536043617632e-05},
  'ugpa': {'ks_stat': 0.02507353013168845, 'p_value': 0.5780486463150973},
  'bar': {'ks_stat': 0.09095966123268218, 'p_value': 2.599780285172037e-07}},
 'correlation_diff': 0.04110556994687435}

For numerical columns, The KSComplement returns 1-(KS statistics) where the Kolmogorov Smirnov (KS) statistic is the maximal difference between the CDF of the variable in the original and the synthetic data. A value close to 1 represents a better fit.

For categorical columns, The TVComplement returns 1 - (TV Distance), where the Total Variation Distance measures the relative frequencies of the categories of a given variable in the original and the synthetic data. A value close to 1 represents a better fit